
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://moodle.df.senac.br/faculdade/pluginfile.php/1/theme_lambda/logo/1716924091/Logo-SENAC-PNG.png" style="width: 300px; height: 150px">
</div>

#Usando Spark SQL para criar Views e Tabelas



## Criando _views_ temporárias para trabalhar com dados em formato tabular

Em _pipelines_ de engenharia de dados, podemos criar _views_ temporárias para transformações dos dados e então persistência nas tabelas finais.

Vamos usar dessa técnica para trabalhar com os dados em formato tabula, mais simples de abstrair.

O código abaixo, cria uma _temp view_ com valores literais.

Execute as células para ver o resultado:

Versao de ciação de um temp view com Python e dentro SQL

In [0]:
spark.sql("""
    CREATE OR REPLACE TEMP VIEW teste(nome, valor) AS VALUES
    ("João", 1),
    ("Maria", 2),
    ("José", 3)
    """)

spark_df = spark.sql("select * from teste")

display(spark_df.select(spark_df.nome, spark_df.valor))

nome,valor
João,1
Maria,2
José,3


Versao de ciação de um temp view com SQL

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW teste_sql(nome, valor) AS VALUES
("João", 1),
("Maria", 2),
("José", 3); -- Lembre que SQL não é Python! Múltiplas linhas de comando em uma mesma célula, utilize ; no final do comando

select * from teste_sql;

nome,valor
João,1
Maria,2
José,3


Você pode criar _views_ com base em uma consulta Spark SQL também:


In [0]:
# Exemplo
spark.sql("""
    CREATE OR REPLACE TEMP VIEW nome_view(campo_1, campo_2 , ... , campo_n ) AS 
        SELECT campo_1, campo_2 , ... , campo_n  FROM read_files(
        'caminho_arquivo',
        -- demais opções)
    """)

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-338097625564229>:2
      1 # Exemplo
----> 2 spark.sql("""
      3     CREATE OR REPLACE TEMP VIEW nome_view(campo_1, campo_2 , ... , campo_n ) AS 
      4         SELECT campo_1, campo_2 , ... , campo_n  FROM read_files(
      5         'caminho_arquivo',
      6         -- demais opções)
      7     """)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1387, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1385 try:
   1386     litArgs = {k: _t


### Atividade

Utilizando o arquivo _clickstream-ptwiki-2024-06.tsv_, crie uma _temp view_ chamada _clickstream_, com as seguintes colunas: prev, curr, type, n

Então responda as seguintes questões, utilizando PySpark ou Spark SQL:

1. Quantas linhas há no arquivo/view?

1. Quais são os 10 primeiros pares (prev, curr) com maior número de ocorrências?

1. Quais são os 10 primeiros pares (prev, curr) com maior número de ocorrências, sendo o campo 'type' = 'external'?

1. Qual é o artigo mais redirecionado pelos sites de busca (prev = other-search)?

1. Qual é o artigo mais navegado tendo como origem outros artigos do próprio Wikipedia?

**Dica**: a clásula `LIMIT` pode ajudar

In [0]:
spark.sql("""
    CREATE OR REPLACE TEMP VIEW clickstream(prev, curr, type, n) AS 
       SELECT * FROM read_files('dbfs:/Volumes/senac/default/clickstream/clickstream-ptwiki-2024-06.tsv',
        format => 'csv', 
        delimiter => '\t', 
        header => 'false', 
        schemaEvolutionMode => 'none');  
        """)

spark_df = spark.sql("select * from clickstream")

display(spark_df.select(spark_df.prev, spark_df.curr,spark_df.type, spark_df.n))

display(spark_df.count())


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW clickstream(prev, curr, type, n) AS 
       SELECT * FROM read_files('dbfs:/Volumes/senac/default/clickstream/clickstream-ptwiki-2024-06.tsv',
        format => 'csv', 
        delimiter => '\t', 
        header => 'false', 
        schemaEvolutionMode => 'none'); 

select * from clickstream;


Quantas linhas há no arquivo/view?

In [0]:
%sql
select count(*) as Total_de_linhas
from clickstream;

Quais são os 10 primeiros pares (prev, curr) com maior número de ocorrências?

In [0]:
%sql
SELECT prev, curr, n AS num_ocorrencias
FROM clickstream
ORDER BY num_ocorrencias DESC
LIMIT 10;

Quais são os 10 primeiros pares (prev, curr) com maior número de ocorrências, sendo o campo 'type' = 'external'?

In [0]:
%sql
SELECT prev, curr, type, n AS num_ocorrencias
FROM clickstream
WHERE type = 'external'
ORDER BY num_ocorrencias DESC
LIMIT 10;

Qual é o artigo mais redirecionado pelos sites de busca (prev = other-search)?

In [0]:
%sql
SELECT curr, type, n AS num_ocorrencias
FROM clickstream
WHERE prev = 'other-search'
ORDER BY num_ocorrencias DESC
LIMIT 1;

Qual é o artigo mais navegado tendo como origem outros artigos do próprio Wikipedia?

In [0]:
%sql
SELECT curr, type, n AS num_ocorrencias
FROM clickstream
WHERE type = 'link'
ORDER BY num_ocorrencias DESC
LIMIT 1;